# Project scoping 

## Description

The following notebook details our approach to extract valuable data from a retail dataset.

1. We will first start by exploring and understanding the data at hand to extract valuable insight.

2. We will profile and clean the data.

3. We will experiment a few clustering algorithms to look for meaningful groups of customers.

## Data Collection

We will base this analysis on the three given sheets : 
* Customer : Customer information including demographics (~ 127 KB) 

* Transaction : Transaction of customers (1,41 MB)

* Product Hierarchy : Product information (588 B)

# Import libraries

In [ ]:
# For data manipulation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from pandas_profiling import ProfileReport

# For interaction with OS
import os 

# Data Exploration

## Import Data

* Let's load the data into a Pandas dataframe :

In [ ]:
INPUT_DIR = '/kaggle/input/retail-case-study-data/'

PATH_CUSTOMER = INPUT_DIR + 'Customer.csv'
PATH_TRANSACTION = INPUT_DIR + 'Transactions.csv'
PATH_PROD_CAT = INPUT_DIR + 'prod_cat_info.csv'

df_customer = pd.read_csv(PATH_CUSTOMER)
df_transaction = pd.read_csv(PATH_TRANSACTION)
df_prod_cat = pd.read_csv(PATH_PROD_CAT)

In [ ]:
def getDataframeInfo(df: pd.DataFrame, nb_row: int) -> pd.DataFrame:
    print(df.shape)
    print(df.columns)
    return df.head(nb_row)

* Customer 

In [ ]:
getDataframeInfo(df_customer, 3)

* Transaction

In [ ]:
getDataframeInfo(df_transaction, 3)

In [ ]:
getDataframeInfo(df_prod_cat, 3)

* Let's unify the column names to simplify further join operations.

In [ ]:
# Lowercase the column names and rename them when needed
def get_unified_table(df: pd.DataFrame, cols : dict = {}) -> pd.DataFrame:
    
    df = df.rename(columns=str.lower)
    
    if cols: df = df.rename(columns=cols)
    return df
    

df_customer = get_unified_table(df_customer, {'customer_id':'cust_id'})
df_transaction = get_unified_table(df_transaction, {'qty':'quantity'})
df_prod_cat = get_unified_table(df_prod_cat, {'prod_sub_cat_code':'prod_subcat_code'})

In [ ]:
df_customer.head(3)

In [ ]:
df_transaction.head(3)

In [ ]:
df_prod_cat.head(3)

## Create New Dataframe

* We create a new dataframe **df** by joining the three input dataframes.

### Create prod_trans

First, let's create **df_prod_trans** combining **df_transaction** and **df_prod_cat** data

In [ ]:
df_prod_trans = df_transaction.merge(df_prod_cat,
                         on=['prod_cat_code', 'prod_subcat_code'],
                         how='left')


Let's look at the shape of our data :

In [ ]:
df_prod_trans.shape

We have 23 053 rows and 12 columns to explore.

* Let's see how many null values we have per column.

In [ ]:
df_prod_trans.isnull().sum()

* No null value found.

### Merge prod_trans with customer

In [ ]:
df_final = df_prod_trans.merge(df_customer,
                              on=['cust_id'],
                              how='left')

In [ ]:
df_final.head(3)

* Let's compare the shape of our final dataframe vs the df_prod_trans

In [ ]:
df_final.shape

In [ ]:
df_prod_trans.shape

* There is the same number of rows in df_final and df_prod_trans, meaning that all the product transactions are present in the final table.  

In [ ]:
df_final.dtypes

* Let's check how many null values we have per column.

In [ ]:
df_final.isnull().sum()

* Let's convert object dtype to dates (YYYY-mm-dd)

In [ ]:
df_final[['dob', 'tran_date']].head(3)

In [ ]:
df_final['dob'] = pd.to_datetime(df_final["dob"], format='%d-%m-%Y')
df_final['tran_date'] = pd.to_datetime(df_final["tran_date"])

In [ ]:
df_final[['dob', 'tran_date']].head(3)

## Data Cleaning

### Missing values

* Let's have a look at the percentage of missing values in each column to decide whether some columns should be dropped.

In [ ]:
percent_missing = df_final.isnull().sum() * 100 / len(df_final)
missing_values = pd.DataFrame({'col_name': df_final.columns,
                              'percent_missing' : percent_missing})
missing_values

* Luckily, missing values only represents a very tiny part of the whole dataset. All the columns have less than 80% missing values, meaning that we don't have to drop any of them.

## Duplicated values

In [ ]:
df_final.duplicated().sum()

* Let's drop the duplicate rows()

In [ ]:
df_final = df_final.drop_duplicates()

In [ ]:
df_final.duplicated().sum()

### Correlation

* Let's take a look at the correlation between the numerical columns :

In [ ]:
corr = df_final.corr()

plt.subplots(figsize=(20,15))
ax = sns.heatmap(corr,
                 vmin=-1, vmax=1, center=0,
                 cmap=sns.diverging_palette(10, 220, n=2000),
                 linewidths=.5)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right')

* There is no column correlated perfectly correlated to another, so we don't drop any columns.

### Data inspection

In [ ]:
df_final.info()

In [ ]:
df_final.head(5)

In [ ]:
df_final.tail(5)

* Let's use the transaction_id as index

In [ ]:
df_final.set_index('transaction_id')

Let's look at the *Five number summary* :
- the sample minimum (smallest observation)
- the lower quartile or first quartile
- the median (the middle value)
- the upper quartile or third quartile
- the sample maximum (largest observation)

In [ ]:
df_final.describe()

- We can already see from the description that some columns have negative values that are actually returns.
Let's get rid of negative quantities, they should remove at the same time all the other negative values.

In [ ]:
df_no_ret = df_final[(df_final['quantity']) > 0 ]

* Let's check that there are no negative values anymore

In [ ]:
df_no_ret.describe()

In [ ]:
df_no_ret.set_index('transaction_id')

* Now that only actual purchases remain,let's check the columns for irregularities :

#### 'quantity'

- Let's start with ***'quantity'***.

In [ ]:
fig = px.box(df_no_ret, y="quantity")
fig.show()

* At first glance, the boxplot doesn't show any outliers. Let's look at the distribution.

In [ ]:
fig = px.histogram(df_no_ret, x="quantity", range_x=(0,5))
fig.show()

#### 'rate'

* Now let's look at ***'rate'***:


In [ ]:
df_no_ret['rate'].describe()

In [ ]:
fig = px.box(df_no_ret, y="rate")
fig.show()

In [ ]:
fig = px.histogram(df_no_ret, x="rate", range_x=(0,1400))
fig.show()

In [ ]:
df_no_ret['rate'].value_counts(normalize=True) * 100

#### Tax

* Let's have a look at ***'tax'***

In [ ]:
fig = px.box(df_no_ret, y="tax")
fig.show()

* The boxplot show some outliers, above 700. Let's take a look at these data

In [ ]:
df_no_ret[df_no_ret['tax'] > 700].head()

* Without the business context, we can't verify this, but we suppose these high tax are real ones, applied to some expensive prod categories as shown below : Electronics, Home and kitchen, some clothing... Let's keep these outliers.

#### total_amt

In [ ]:
fig = px.box(df_no_ret, y="total_amt")
fig.show()

#### city_code

In [ ]:
fig = px.box(df_no_ret, y="city_code")
fig.show()

# Feature Engineering

# Machine Learning

# Conclusion & Insights

- Let's check the most popular categorical variables by describing the dataframe.

In [ ]:
df_final.loc[:,df_final.dtypes=="object"].describe()

In [ ]:
df_final.head()

* Let's look at the distribution for the following columns :
    - gender
    - store_type
    - prod_cat
    - prod_subcat
    - city_code

#### gender

In [ ]:
plt.figure(figsize=(5,5))
#sns.countplot(df_no_ret['gender'])
df_no_ret.groupby('gender')['cust_id'].count().sort_values(ascending=True).plot(kind='barh')
plt.show()

* Men represents the biggest part of the customers, although both gender are well represented.

#### store_type

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(df_no_ret['store_type'])
plt.xlabel('Store Type')
plt.show()

* Half of the total purchases are made via eShop, while the remaining types are slightly equally distributed.

#### prod_cat

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(df_no_ret['prod_cat'])
plt.xlabel('Product Category')
plt.show()

* The top selling categories are in order : Books, Electronics and Home and kitchen. Then, followede by Footwear, Clothing and Bags.

#### prod_subcat

In [ ]:
plt.figure(figsize=(8,8))
df_no_ret.groupby('prod_subcat')['prod_subcat'].count().plot(kind='barh')
plt.xlabel('Count')
plt.ylabel('Product Subcategory')
plt.show()

#### city_code

In [ ]:
plt.figure(figsize=(5,5))
df_no_ret.groupby('city_code')['cust_id'].count().sort_values(ascending=True).plot(kind='barh')
plt.xlabel('Count')
plt.ylabel('City Code')
plt.show()

In [ ]:
distrib_cols = ['prod_subcat_code', 'prod_cat_code', 'quantity', 'rate', 'tax', 'total_amt']
df_distrib = df_no_ret.loc[:,distrib_cols]

In [ ]:
for col in df_distrib.columns:
    df_no_ret[col].plot(kind='hist')
    plt.title(col)
    plt.show()

In [ ]:
df_no_ret = df_no_ret.set_index('transaction_id')

In [ ]:
df_no_ret.head()

In [ ]:
profile = ProfileReport(df_no_ret, title="Pandas Profiling Report")

In [ ]:
profile.to_widgets()

In [ ]:
profile.to_notebook_iframe()

In [ ]:
profile.to_file("/kaggle/working/report.html")